# Exercise 1 - Basic prompt engineering
The goal of this exercise is to become familiar with the basic prompt engineering techniques.
We do this by first interactively exploring the examples from the slides.
Then, we will apply the same techniques to extract information from a dataset of housing descriptions.

Before we start, let's import the necessary libraries and create a client for the Azure OpenAI API.

In [ ]:
from llm_in_production.openai_utils import get_openai_client
import dotenv
import os
import pandas as pd


dotenv.load_dotenv()
# Here we create the client.
client = get_openai_client()

## Exercise 1a: Examples from the slides
In this exercise, you can interactively re-run the examples from the slides.
We have implemented these examples using both the text completion and chat API.


For each of the examples, do the following:
- First, just run the example and see what happens.
- Then, compare the prompts for text completion and chat API. How are they different?
- Afterwards, modify the prompt to see how it affects the output.
- Optionally, you can try to give it a different problem and see if it still works.


### Classification
Here we show a prompt that transforms a LLM into a classifier.

#### Classification with the *Text completion* API
Using the text completion API, the prompt looks as follows:

In [ ]:
text = """
Classify the text into neutral, negative, or positive.
Output format: one of the above classes.

Text: I think the food was okay.
Sentiment: """

response = client.completions.create(
    model=os.environ["GPT_35_TURBO_INSTRUCT_MODEL_NAME"],
    prompt=text,
)
completion = response.choices[0].text.strip()
print(completion)

#### Classification with the *Chat* API
Using the chat API, the prompt looks as follows:

In [ ]:
system_prompt = "Classify the user messages into neutral, negative, or positive. Respond only with one of the above classes."
text = "I think the food was okay."
response = client.chat.completions.create(
    model=os.environ["GPT_35_CHAT_MODEL_NAME"],
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": text},
    ],
)
response.choices[0].message.content

### Text summarization
We can also use the LLM to summarize text. Here we show an example of summarization prompt.

#### Text summarization with the *Text completion* API
Using the text completion API, the prompt looks as follows:

In [ ]:
text = """
Extract the names of locations in the following text.
Desired format:
Places: Comma separated list of location_names
Input: In Paris, love's embrace ignites the night, Venice's canals whisper secrets in moonlight, Berlin's walls echo stories of resilience and might
Places: """

response = client.completions.create(
    model=os.environ["GPT_35_TURBO_INSTRUCT_MODEL_NAME"],
    prompt=text,
    temperature=0.0,
)
completion = response.choices[0].text.strip()
places = completion.split(",")
print(places)

#### Text summarization with the *Chat* API
Using the chat API, the prompt looks as follows:

In [ ]:
system_prompt = """
Extract the names of locations from the user message.
Response format: Comma separated list of location_names
"""

user_message = "In Paris, love's embrace ignites the night, Venice's canals whisper secrets in moonlight, Berlin's walls echo stories of resilience and might"

response = client.chat.completions.create(
    model=os.environ["GPT_35_CHAT_MODEL_NAME"],
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message},
    ],
    temperature=0.0,
)

completion = response.choices[0].message.content.strip()
places = completion.split(",")
print(places)

### Chain of thought reasoning
For more complex reasoning tasks, we can use the [chain of thought reasoning technique](https://www.promptingguide.ai/techniques/cot#chain-of-thought-cot-prompting). In this prompt, the LLM first generates a thought by writing down its observations, reasoning, and calculations. Then, it writes down the answer after the `Answer:`.

#### Chain of thought reasoning with the *Text completion* API
Using the text completion API, the prompt looks as follows:

In [ ]:
text = """
You solve mathematical problems by writing down your reasoning and calculations and then writing down the answer.

Respond using this format:
Thought: describe here you write reasoning, and calculations need to solve the problem
Answer: your answer without any explanation, text or calculations.


Problem: I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1.
Thought:""".strip()

response = client.completions.create(
    model=os.environ["GPT_35_TURBO_INSTRUCT_MODEL_NAME"],
    prompt=text,
    temperature=0.0,
    max_tokens=1024
)
completion = response.choices[0].text.strip()
print(completion)
answer = completion.split("Answer:")[1].strip()
print("#" * 80)
print("Extracted answer:", answer)

#### Chain of though reasoning with the *Chat* API
Using the chat API, the prompt looks as follows:

In [ ]:
system_prompt = """
The user will give you mathematical problems to solve.
You should solve the problems and respond with the answer.
Respond using this format:
Thought: describe here you write reasoning, and calculations need to solve the problem
Answer: your answer without any explanation, text or calculations.
""".strip()

user_message = """
I went to the market and bought 10 apples. 
I gave 2 apples to the neighbor and 2 to the repairman. 
I then went and bought 5 more apples and ate 1.
How many apples did I remain with?
""".strip()

response = client.chat.completions.create(
    model=os.environ["GPT_35_CHAT_MODEL_NAME"],
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message},
    ],
    temperature=0.0,
)
completion = response.choices[0].message.content.strip()
# Here we extract the answer from the completion by splitting the text on the Answer: keyword
# and taking the part after the keyword.
answer = completion.split("Answer:")[1].strip()
print(completion)
print("#" * 80)
print("Extracted answer:", answer)

### Few shot examples
Few shot examples are another useful technique to guide the LLM to generate a specific output.
It is especially useful when it easier to just show the desired format, instead of describing it.

In the Chat API, we can implement this by writing our own `assistant` messages. This prompt looks as follows:

In [ ]:
response = client.chat.completions.create(
    model=os.environ["GPT_35_CHAT_MODEL_NAME"],
    messages=[
        {"role": "system", "content": "Posible values for sentiment: neutral, negative, good"},
        {"role": "user", "content": "I think the food was okay."},
        {"role": "assistant", "content": "Sentiment: neutral"},
        {"role": "user", "content": "I think the food was bad."},
        {"role": "assistant", "content": "Sentiment: negative"},
        {"role": "user", "content": "I think the food was good."},
    ],
    temperature=0.0,
)
completion = response.choices[0].message.content.strip()
print(completion)

## Extracting housing information
Now that we have seen some examples of prompt engineering, let's apply them to extract information from a dataset of housing descriptions. 

We have a dataset with house listing descriptions. This description contains information about the house, such as the number of bedrooms, the neighborhood, and whether pets are allowed. However, everybody writes the information slightly differently, making it hard to parse it using regular expressions. Let's try to use the LLM to extract this information.

We have also included the known answer for each example in the dataset. This makes it easier to evaluate your solution, although in practice, we would not have this information.


In [ ]:
df = pd.read_csv("houses.csv")
print(df.shape)
df

As you can see, we have the following information available:

In [ ]:
house_idx = 0
row = df.iloc[house_idx]
print("House:", house_idx)
print("City:", row["city"])
print("Price:", row["price"])
print("Surface area:", row["surface_area"])
print("Bedrooms:", row["bedrooms"])
print("Description:\n", row["description"])

## Exercise 1b: Extract the number of bedrooms
The first thing we want to extract is the number of bedrooms.
This should be a positive integer number.
It is your task to write a prompt that extracts this information from the description in the function below.

For example:
- Description: "The house has 3 bedrooms and two bathrooms."
- Extracted number: "3"

If you run this cell, it will automatically test your function against the known correct answers in the dataset.

In [ ]:
def extract_n_bedrooms(description: str) -> str:
    # YOUR CODE HERE START
    system_prompt = """
    The user sends a description of a house. Extract the number bedrooms room from the description.
    Respond only with the an integer number for the number bedrooms room from the description. 
    Do not include any other text or other information like explanation in the response.
    """
    # YOUR CODE HERE END
    
    # Feel free to change more of the code below if you want to
    # This is just a skeleton to get you started
    response = client.chat.completions.create(
        model=os.environ["GPT_35_CHAT_MODEL_NAME"],
        messages=[
            {"role": "system", "content": system_prompt},
            # YOUR CODE HERE START: Optionally add some few shot examples here
            {"role": "user", "content": "The house has 3 bedrooms and two bathrooms."},
            {"role": "assistant", "content": "3"},
            # YOUR CODE HERE END
            {"role": "user", "content": description},
        ],
        temperature=0.0,
    )
    return response.choices[0].message.content


# Here we test your function against the known correct answers
# Feel free to inspect df["bedrooms"] to get a better understanding go the answers.
for i, row in df.iterrows():
    n_bedrooms = extract_n_bedrooms(row["description"])
    assert n_bedrooms.isdigit() or isinstance(n_bedrooms, float), f"For row {i}, we got not a number: {n_bedrooms}"
    assert row["bedrooms"] == int(n_bedrooms), f"For row {i}, we expected {row['bedrooms']} but got {n_bedrooms}"
    print(f"✅ Row {i} passed `{n_bedrooms}` == `{row['bedrooms']}`")

## Exercise 1c: Extract neighborhood name
The second thing we want to extract is the name of the neighborhood.
This should be a string with the name of the neighborhood.
It is your task to write a prompt that extracts this information from the description in the function below.

For example:
- Description: "The house is located in De Pijp."
- Extracted neighborhood: "De Pijp"

If you run this cell, it will automatically test your function against the known correct answers in the dataset.

In [ ]:
def extract_neighborhood(description):
    # YOUR CODE HERE START
    system_prompt = """
    The user sends a description of a house. Extract the name of the neighborhood from the description.
    Respond only with the name of the neighborhood, do not include any other text or other information like the city name.
    Respond with unknown if the neighborhood name is not mentioned in the description.
    No trailing spaces or punctuation.
    """
    # YOUR CODE HERE END
    
    # Feel free to change more of the code below if you want to
    # This is just a skeleton to get you started
    response = client.chat.completions.create(
        model=os.environ["GPT_35_CHAT_MODEL_NAME"],
        messages=[
            {"role": "system", "content": system_prompt},
             # YOUR CODE HERE START: Optionally add some few shot examples here
             # YOUR CODE HERE END
            {"role": "user", "content": description},
        ],
        temperature=0.0,
    )
    message = response.choices[0].message.content
    
    return message.replace(".", "").strip()

# Here we test your function against the known correct answers
# Feel free to inspect df["neighborhood"] to get a better understanding go the answers.
for i, row in df.iterrows():
    neighborhood_name = extract_neighborhood(row["description"])
    assert neighborhood_name.lower().strip() == row["neighborhood"].lower().strip(), f"For row {i}, we expected `{row['neighborhood']}` but got `{neighborhood_name}`"
    print(f"✅ Row {i} passed {neighborhood_name} == {row['neighborhood']}")
    

## Exercise 1d: Extract if pets are allowed
The last thing we want to extract is whether pets are allowed.
This can be one of the following classes:
- `allowed`: If the description explicitly mentions that pets are allowed.
- `not_allowed`: If the description explicitly mentions that pets are not allowed.
- `unknown`: If the description does not explicitly mention whether pets are allowed or not.

It is your task to write a prompt that extracts this information from the description in the function below.

For example:
- Description: "No dogs allowed."
- Extracted: "not_allowed"

If you run this cell, it will automatically test your function against the known correct answers in the dataset.

In [ ]:
def are_pets_allowed(description):
    # YOUR CODE HERE START
    system_prompt = """
    The user sends a description of a house. Classify the text with following lables:
    - allowed: If the description explicitly mentions that pets are allowed.
    - not_allowed: If the description explicitly mentions that pets are not allowed.
    - unknown: If the description does not explicitly mention whether pets are allowed or not.

    Format your response as follows:
    Thought: describe here your reasoning.
    Answer: one of the above classes
    """
    # YOUR CODE HERE END

    response = client.chat.completions.create(
        model=os.environ["GPT_35_CHAT_MODEL_NAME"],
        messages=[
            {"role": "system", "content": system_prompt},
            # YOUR CODE HERE START: Optionally, add some few shot examples here
            # YOUR CODE HERE END
            {"role": "user", "content": description},
        ],
        temperature=0.0,
    )
    message = response.choices[0].message.content
    # YOUR CODE HERE START: Parse the response
    return message.lower().split("answer:")[1].strip().replace(".", "")
    # YOUR CODE HERE END

# Here we test your function against the known correct answers
# Feel free to inspect df["neighborhood"] to get a better understanding go the answers.
for i, row in df.iterrows():
    pets_allowed = are_pets_allowed(row["description"])
    assert pets_allowed in ["allowed", "not_allowed", "unknown"], f"For row {i}, we got unexpected class: {pets_allowed}"
    assert pets_allowed == row["pets_allowed"], f"For row {i}, we expected {row['pets_allowed']} but got {pets_allowed}"
    print(f"✅ Row {i} passed {pets_allowed} == {row['pets_allowed']}")


## Exercise 1e: Reflection
So far, we have implemented multiple functions that each extract some information from the description.
Before we continue, let's reflect on the current approach.
- What are the disadvantages of the current approach with separate functions for each extraction task? 
- How could we improve the current approach?

## Optional exercise 1f: Combine all the extraction methods into a single prompt
Let's try to combine all the extraction methods into a single function.
It is your task to write a prompt that extracts this information from the description in the function below.

For example:
- Description: "This is a 3 bedrooms house in De Pijp where pets are not allowed."
- Extracted: `{"n_bedrooms": 3, "neighborhood": "De Pijp", "pets_allowed": "not_allowed"}`

If you run this cell, it will automatically test your function against the known correct answers in the dataset.

Note: might need the [json.load](https://docs.python.org/3/library/json.html) function to parse a JSON string into a Python dictionary.

Note: In the Chat API, setting `response_format={"type": "json_object"}` guarantees the ouput is a JSON (although it doesn't guarantee it will be the format we want!).

In [ ]:
import json
from typing import Any
def extract_info(description: str) -> dict[str, Any]:
    # YOUR CODE HERE START
    system_prompt = """
    The user sends a description of a house. Extract the number bedrooms room, the name of the neighborhood, and whether pets are allowed from the description.

    Your response should be a JSON object with the following format:
    {
    “n_bedrooms”: … # a positive integer no other text
    “neighborhood”: … # Name of the neighborhood no other text
    “pets_allowed”: … # allowed/not_allowed/unknown Use unknown if the description does not explicitly mention whether pets are allowed or not.
    }
    """.strip()
    # YOUR CODE HERE END
    response = client.chat.completions.create(
        model=os.environ["GPT_35_CHAT_MODEL_NAME"],
        messages=[
            {"role": "system", "content": system_prompt},
            # YOUR CODE HERE START: Optionally add some few shot examples here
            # YOUR CODE HERE END
            {"role": "user", "content": description},
        ],
        response_format={"type": "json_object"},
        temperature=0.0,
    )
    
    # YOUR CODE HERE START: Parse the response
    data = json.loads(response.choices[0].message.content)
    # YOUR CODE HERE END
    
    return data

for i, row in df.iterrows():
    info = extract_info(row["description"])
    assert info["n_bedrooms"] == row["bedrooms"], f"For row {i}, we expected {row['bedrooms']} but got {info['n_bedrooms']}"
    assert info["neighborhood"].lower().strip() == row["neighborhood"].lower().strip(), f"For row {i}, we expected `{row['neighborhood']}` but got `{info['neighborhood']}`"
    assert info["pets_allowed"] == row["pets_allowed"], f"For row {i}, we expected {row['pets_allowed']} but got {info['pets_allowed']}"
    print(f"✅ Row {i} passed {info}")